# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse Shipments  \
0  Mar 21 2023 12:20PM  259634        15   9289825   
1  Mar 21 2023 12:20PM  259634        15   9289837   
2  Mar 21 2023 12:20PM  259634        15   9290418   
3  Mar 21 2023 12:20PM  259634        15   9289819   
4  Mar 21 2023 12:20PM  259634        15   9289820   
5  Mar 21 2023 12:20PM  259634        15   9289821   
6  Mar 21 2023 12:20PM  259634        15   9289822   
7  Mar 21 2023 12:20PM  259634        15   9289823   
8  Mar 21 2023 12:20PM  259634        15   9289824   
9  Mar 21 2023 12:20PM  259634        15   9289813   

                          Customer ShipmentStatus  
0  Brookfield Pharmaceuticals, LLC       Released  
1  Brookfield Pharmaceuticals, LLC       Released  
2  Brookfield Pharmaceuticals, LLC       Released  
3  Brookfield Pharmaceuticals, LLC       Released  
4  Brookfield Pharmaceuticals, LLC       Released  
5  Brookfield Pharmaceuticals, LLC       Released  
6  Brookfield Pharmaceuticals, LLC       Released  
7  Brookfield Pharmaceuticals, LLC       Released  
8  Brookfield Pharmaceuticals, LLC       Released  
9  Brookfield Pharmaceuticals, LLC       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
26  259644       Released          1
27  259645       Released          5
28  259646       Released          1
29  259647       Released          1
30  259648       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
259644                NaN        NaN       1.0
259645                NaN        NaN       5.0
259646                NaN        NaN       1.0
259647                NaN        NaN       1.0
259648                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
259574                0.0        1.0       0.0
259596                0.0        1.0       0.0
259599                0.0        2.0       0.0
259606               16.0       30.0      27.0
259609                0.0        0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
259574                  0          1         0
259596                  0          1         0
259599                  0          2         0
259606                 16         30        27
259609                  0          0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               259574          0          1         0
1               259596          0          1         0
2               259599          0          2         0
3               259606         16         30        27
4               259609          0          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               259574                   1         
1               259596                   1         
2               259599                   2         
3               259606        16        30       27
4               259609                            1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                         Customer
0   Mar 21 2023 12:20PM  259634        15  Brookfield Pharmaceuticals, LLC
32  Mar 21 2023 12:10PM  259648        16                  SHL Pharma, LLC
33  Mar 21 2023 12:06PM  259647        19           GCH Granules USA, Inc.
34  Mar 21 2023 11:57AM  259646        18             Emersa Waterbox, LLC
35  Mar 21 2023 11:50AM  259645        10                 Eywa Pharma Inc.
40  Mar 21 2023 11:46AM  259644        10                 Eywa Pharma Inc.
41  Mar 21 2023 11:45AM  259643        19              AdvaGen Pharma, Ltd
42  Mar 21 2023 11:40AM  259642        19           GCH Granules USA, Inc.
43  Mar 21 2023 11:39AM  259641        10              MedStone Pharma LLC
56  Mar 21 2023 11:39AM  259640        10              MedStone Pharma LLC

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                         Customer  \
0  Mar 21 2023 12:20PM  259634        15  Brookfield Pharmaceuticals, LLC   
1  Mar 21 2023 12:10PM  259648        16                  SHL Pharma, LLC   
2  Mar 21 2023 12:06PM  259647        19           GCH Granules USA, Inc.   
3  Mar 21 2023 11:57AM  259646        18             Emersa Waterbox, LLC   
4  Mar 21 2023 11:50AM  259645        10                 Eywa Pharma Inc.   
5  Mar 21 2023 11:46AM  259644        10                 Eywa Pharma Inc.   
6  Mar 21 2023 11:45AM  259643        19              AdvaGen Pharma, Ltd   
7  Mar 21 2023 11:40AM  259642        19           GCH Granules USA, Inc.   
8  Mar 21 2023 11:39AM  259641        10              MedStone Pharma LLC   
9  Mar 21 2023 11:39AM  259640        10              MedStone Pharma LLC   

  Completed Executing Released  
0                           32  
1                            1  
2                            1  
3                            1  
4                            5  
5                            1  
6                            1  
7                            1  
8                           13  
9                            1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                         Customer  \
0  Mar 21 2023 12:20PM  259634        15  Brookfield Pharmaceuticals, LLC   
1  Mar 21 2023 12:10PM  259648        16                  SHL Pharma, LLC   
2  Mar 21 2023 12:06PM  259647        19           GCH Granules USA, Inc.   
3  Mar 21 2023 11:57AM  259646        18             Emersa Waterbox, LLC   
4  Mar 21 2023 11:50AM  259645        10                 Eywa Pharma Inc.   
5  Mar 21 2023 11:46AM  259644        10                 Eywa Pharma Inc.   
6  Mar 21 2023 11:45AM  259643        19              AdvaGen Pharma, Ltd   
7  Mar 21 2023 11:40AM  259642        19           GCH Granules USA, Inc.   
8  Mar 21 2023 11:39AM  259641        10              MedStone Pharma LLC   
9  Mar 21 2023 11:39AM  259640        10              MedStone Pharma LLC   

  Released Executing Completed  
0       32                      
1        1                      
2        1                      
3        1                      
4        5                      
5        1                      
6        1                      
7        1                      
8       13                      
9        1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                         Customer  \
0  Mar 21 2023 12:20PM  259634        15  Brookfield Pharmaceuticals, LLC   
1  Mar 21 2023 12:10PM  259648        16                  SHL Pharma, LLC   
2  Mar 21 2023 12:06PM  259647        19           GCH Granules USA, Inc.   
3  Mar 21 2023 11:57AM  259646        18             Emersa Waterbox, LLC   
4  Mar 21 2023 11:50AM  259645        10                 Eywa Pharma Inc.   

  Released Executing Completed  
0       32                      
1        1                      
2        1                      
3        1                      
4        5

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                         Customer  \
0  Mar 21 2023 12:20PM  259634        15  Brookfield Pharmaceuticals, LLC   
1  Mar 21 2023 12:10PM  259648        16                  SHL Pharma, LLC   
2  Mar 21 2023 12:06PM  259647        19           GCH Granules USA, Inc.   
3  Mar 21 2023 11:57AM  259646        18             Emersa Waterbox, LLC   
4  Mar 21 2023 11:50AM  259645        10                 Eywa Pharma Inc.   

   Released  Executing  Completed  
0      32.0        NaN        NaN  
1       1.0        NaN        NaN  
2       1.0        NaN        NaN  
3       1.0        NaN        NaN  
4       5.0        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                         Customer  \
0  Mar 21 2023 12:20PM  259634        15  Brookfield Pharmaceuticals, LLC   
1  Mar 21 2023 12:10PM  259648        16                  SHL Pharma, LLC   
2  Mar 21 2023 12:06PM  259647        19           GCH Granules USA, Inc.   
3  Mar 21 2023 11:57AM  259646        18             Emersa Waterbox, LLC   
4  Mar 21 2023 11:50AM  259645        10                 Eywa Pharma Inc.   

   Released  Executing  Completed  
0      32.0        0.0        0.0  
1       1.0        0.0        0.0  
2       1.0        0.0        0.0  
3       1.0        0.0        0.0  
4       5.0        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         3375144      53.0        7.0        0.0
15          519266      44.0       13.0        0.0
16          259648       1.0        0.0        0.0
18          519255       2.0        0.0        0.0
19         1817392      32.0       32.0       16.0
20          259623       2.0       30.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  3375144      53.0        7.0        0.0
1        15   519266      44.0       13.0        0.0
2        16   259648       1.0        0.0        0.0
3        18   519255       2.0        0.0        0.0
4        19  1817392      32.0       32.0       16.0
5        20   259623       2.0       30.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      53.0        7.0        0.0
1        15      44.0       13.0        0.0
2        16       1.0        0.0        0.0
3        18       2.0        0.0        0.0
4        19      32.0       32.0       16.0
5        20       2.0       30.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   53.0
1        15  Released   44.0
2        16  Released    1.0
3        18  Released    2.0
4        19  Released   32.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10    15   16   18    19    20
Status                                     
Completed   0.0   0.0  0.0  0.0  16.0   0.0
Executing   7.0  13.0  0.0  0.0  32.0  30.0
Released   53.0  44.0  1.0  2.0  32.0   2.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10    15   16   18    19    20
0          Completed   0.0   0.0  0.0  0.0  16.0   0.0
1          Executing   7.0  13.0  0.0  0.0  32.0  30.0
2           Released  53.0  44.0  1.0  2.0  32.0   2.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10    15   16   18    19    20
0  Completed   0.0   0.0  0.0  0.0  16.0   0.0
1  Executing   7.0  13.0  0.0  0.0  32.0  30.0
2   Released  53.0  44.0  1.0  2.0  32.0   2.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()